In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('A-track-dataset.csv')

In [5]:
df.head(2)

,Log_Number,payload,label_action
0,0,"GET /forum1_professionnel.asp?n=/.\\\""./.\\\""....",System_Cmd_Execution
1,1,POST /owa/auth/logon.aspx?replaceCurrent=1%22%...,System_Cmd_Execution


In [6]:
# df.drop('Log_Number', axis=1 ,inplace=True)

In [7]:
df.columns

Index(['Log_Number', 'payload', 'label_action'], dtype='object')

데이터 전처리

In [8]:
import urllib.parse  #URL 처리와 관련된 여러 유용한 함수와 클래스 제공

def unquote(text):
    k = 0               #반복 회수를 세기 위한 변수 k
    uq_prev = text      #uq_prev text 인자 받은거 입력
    while(k < 100):     #반복 작업을 100회 이내로 실행
        uq = urllib.parse.unquote_plus(uq_prev)
        if uq == uq_prev:
            break
        else:
            uq_prev = uq
    
    return uq_prev

def remove_new_line(text):
    text = text.strip()
    text = ' '.join(text.splitlines())
    
    return text

def remove_multiple_whitespaces(text):
    return ' '.join(text.split())

def clean_pattern(pattern):
    pattern = unquote(pattern)                      #url decoding
    pattern = remove_new_line(pattern)              #\n 줄바꿈 문자 제거
    pattern = pattern.lower()                       #소문자로 바꾸는거
    pattern = remove_multiple_whitespaces(pattern)  # 공백 제거
    
    return pattern

In [9]:
# df.loc[:, "payload"] = df["payload"].apply(clean_pattern)
df.loc[:, "payload"] = df["payload"].apply(clean_pattern)

In [10]:
import sklearn # Check Sklearn version
sklearn.__version__

'1.3.0'

In [11]:
import pandas as pd
import numpy as np

In [12]:
# df = pd.read_csv('../dataset/A-Track-clean.csv')

In [13]:
df.head(2)

,Log_Number,payload,label_action
0,0,"get /forum1_professionnel.asp?n=/.\\\""./.\\\""....",System_Cmd_Execution
1,1,"post /owa/auth/logon.aspx?replacecurrent=1"") a...",System_Cmd_Execution


In [14]:
# df.drop('Unnamed: 0', axis=1, inplace=True)

In [15]:
df.columns

Index(['Log_Number', 'payload', 'label_action'], dtype='object')

In [16]:
df.head(2)

,Log_Number,payload,label_action
0,0,"get /forum1_professionnel.asp?n=/.\\\""./.\\\""....",System_Cmd_Execution
1,1,"post /owa/auth/logon.aspx?replacecurrent=1"") a...",System_Cmd_Execution


train-set, test-set split 분리

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score, roc_curve, auc

In [18]:
X_train, X_test, y_train, y_test = train_test_split(
    df['payload'], df['label_action'], test_size=0.2, random_state=42
)


In [19]:
# grid.best_params_
# {'svc__C': 10, 'svc__kernel': 'rbf', 'tfidfvectorizer__ngram_range': (1, 2)}

In [20]:
model = make_pipeline(TfidfVectorizer(input = 'content', lowercase = True, analyzer = 'char', max_features = 1024, ngram_range = (1, 2)), SVC(C = 10, kernel = 'rbf'))

model.fit(X_train, y_train)

Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(analyzer='char', max_features=1024,
                                 ngram_range=(1, 2))),
                ('svc', SVC(C=10))])

In [21]:
model.score(X_test, y_test)

0.8725555555555555

In [22]:
y_pred_test = model.predict(X_test)
test_acc = accuracy_score(y_test,y_pred_test)
test_rep = classification_report(y_test,y_pred_test)

print()
print("---- METRICS RESULTS FOR TESTING DATA ----")
print()
print("Total Rows are: ", X_test.shape[0])
print('[TESTING] Model Accuracy is: ', test_acc)
print('[TESTING] Testing Report: ')
print(test_rep)


---- METRICS RESULTS FOR TESTING DATA ----

Total Rows are:  9000
[TESTING] Model Accuracy is:  0.8725555555555555
[TESTING] Testing Report: 
                               precision    recall  f1-score   support

Automatically_Searching_Infor       0.99      1.00      1.00       226
         Cross_Site_Scripting       0.89      0.82      0.85       267
           Directory_Indexing       0.97      0.95      0.96        76
                    HOST_Scan       0.87      0.87      0.87      1310
           Leakage_Through_NW       1.00      1.00      1.00       198
              Path_Disclosure       0.91      0.87      0.89       965
                SQL_Injection       0.89      0.86      0.87       632
         System_Cmd_Execution       0.82      0.82      0.82      1945
           Vulnerability_Scan       0.87      0.89      0.88      3381

                     accuracy                           0.87      9000
                    macro avg       0.91      0.90      0.90      9000
   

In [23]:
# plot_confusion_matrix(model, X_test, y_test)

In [37]:
# # from sklearn.externals import joblib
# import joblib
# joblib.dump(model, 'tdidf-svc.joblib')